In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *

In [2]:
#Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
assumptions = a_default.copy() # Start with default parameters

## Update parameters for community-selection
assumptions.update({
    'SA': 600*np.ones(3), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 30*np.ones(3), #Number of resources in each class
    'Sgen': 300, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 96,
    "m": 0, # Mortality
    "scale": 10**6,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "l": 0, # Set leakage function to 0 to switch off cross-feeding
    "response": "type III",
    "sigma_max": 5,
    'R0_food': 1000, # Total amount of supplied food
    "rich_medium": True, # Number of food types passed to R0
    # The parameters below will be passed to params_simulation
    "n_propagation": 1, # Length of propagation, or hours within a growth cycle
    "n_transfer": 40, # Number of total transfer, or number of passage
    "n_transfer_selection": 20, # Number of transfer implementing seleciton regimes
    "dilution": 1/1000, # Dilution factor at every transfer
    "n_inoc": 10**6,  #Number of cells sampled from the regional species at start
    "selected_function": "f2_interaction"
})
# Prepare experiment setup in this universe
params, params_simulation = prepare_experiment(assumptions, seed = 1)

In [3]:
params, params_simulation = prepare_experiment(assumptions, seed = 1)
init_state = MakeInitialState(assumptions)
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 
plate.R = make_rich_medium(plate.R, assumptions)
plate.R0 = make_rich_medium(plate.R, assumptions)
plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])
plate = add_community_function(plate, dynamics, assumptions, params_simulation)
# Empty list for saving data
plate_data_list = list() # Plate composition
community_function_list = list() # Community function
assembly_type = 'Trial'
# Run simulation
for i in range(0, 1000):
    plate.Propagate(params_simulation["n_propagation"])
    plate_data = reshape_plate_data(plate, transfer_loop_index = i + 1, assembly_type = assembly_type, community_function_name = 'f1_additive') # Transfer = 0 means that it's before selection regime works upon
    plate_data_list.append(plate_data)
    
    community_function = globals()['f2_interaction'](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = 'f2_interaction', community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
    function_data['Transfer'] =i
    community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon
    transfer_matrix = np.eye(assumptions['n_wells'])
    if ((i+1) % 10 == 0) and ((i+1) % 100 != 0):
        print('Selection' + str(i))
        transfer_matrix = select_top(community_function)
        for k in plate.R0.columns:
            winning_index = np.where(np.sum(transfer_matrix>0,axis=0)>0)[0]
            if k == plate.R0.columns[winning_index][0]:
                continue
            plate.R0[k]  = np.array(plate.R0[plate.R0.columns[winning_index]])
            r_id = np.random.choice(np.where(plate.R0[k]>=0)[0])
            plate.R0[k][r_id] = assumptions['R0_food']/10
            plate.R0 = plate.R0/np.sum(plate.R0) * assumptions['R0_food']
    if (i+1) % 100 ==0:
        transfer_matrix = select_top(community_function)
        m = globals()['directed_selection_migrate'](community_function) 
        plate.N = migrate_from_pool(plate, pool = params_simulation["pool"], migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])
    plate.Passage(transfer_matrix * params_simulation["dilution"])
    cdf = pd.concat(community_function_list)
    cdf.to_csv('data/iterative_interaction_1.csv')

Selection9
Selection19
Selection29
Selection39
Selection49
Selection59
Selection69
Selection79
Selection89
Migration
Selection109
Selection119
Selection129
Selection139
Selection149
Selection159
Selection169
Selection179
Selection189
Migration
Selection209
Selection219
Selection229
Selection239
Selection249
Selection259
Selection269
Selection279
Selection289
Migration
Selection309
Selection319
Selection329
Selection339
Selection349
Selection359
Selection369
Selection379
Selection389
Migration
Selection409
Selection419
Selection429
Selection439
Selection449
Selection459
Selection469
Selection479
Selection489
Migration
Selection509
Selection519
Selection529
Selection539
Selection549
Selection559
Selection569
Selection579
Selection589
Migration
Selection609
Selection619
Selection629
Selection639
Selection649
Selection659
Selection669
Selection679
Selection689
Migration
Selection709
Selection719
Selection729
Selection739
Selection749
Selection759
Selection769
Selection779
Selection789
Migr

In [4]:
params, params_simulation = prepare_experiment(assumptions, seed = 1)
init_state = MakeInitialState(assumptions)
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 
plate.R = make_rich_medium(plate.R, assumptions)
plate.R0 = make_rich_medium(plate.R, assumptions)
plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])
plate = add_community_function(plate, dynamics, assumptions, params_simulation)
# Empty list for saving data
plate_data_list = list() # Plate composition
community_function_list = list() # Community function
assembly_type = 'Trial'
# Run simulation
for i in range(0, 1000):
    plate.Propagate(params_simulation["n_propagation"])
    plate_data = reshape_plate_data(plate, transfer_loop_index = i + 1, assembly_type = assembly_type, community_function_name = 'f1_additive') # Transfer = 0 means that it's before selection regime works upon
    plate_data_list.append(plate_data)
    
    community_function = globals()['f'](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = 'f2_interaction', community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
    function_data['Transfer'] =i
    community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon
    transfer_matrix = np.eye(assumptions['n_wells'])
    if ((i+1) % 10 == 0) and ((i+1) % 100 != 0):
        print('Selection' + str(i))
        transfer_matrix = select_top(community_function)
        for k in plate.R0.columns:
            winning_index = np.where(np.sum(transfer_matrix>0,axis=0)>0)[0]
            if k == plate.R0.columns[winning_index][0]:
                continue
            plate.R0[k]  = np.array(plate.R0[plate.R0.columns[winning_index]])
            r_id = np.random.choice(np.where(plate.R0[k]>=0)[0])
            plate.R0[k][r_id] = assumptions['R0_food']/10
            plate.R0 = plate.R0/np.sum(plate.R0) * assumptions['R0_food']
    if (i+1) % 2000 ==0:
        transfer_matrix = select_top(community_function)
        m = globals()['directed_selection_migrate'](community_function) 
        plate.N = migrate_from_pool(plate, pool = params_simulation["pool"], migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])
    plate.Passage(transfer_matrix * params_simulation["dilution"])
    cdf = pd.concat(community_function_list)
    cdf.to_csv('data/iterative_interaction_2.csv')

KeyError: 'f'